# Task 10 - RandomForest classification

In [2]:
%%bash
export version=`python --version |awk '{print $2}' |awk -F"." '{print $1$2}'`

echo $version

if [ $version == '36' ] || [ $version == '37' ]; then
    echo 'Starting installation...'
    pip3 install pyspark==2.4.8 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
elif [ $version == '38' ] || [ $version == '39' ]; then
    pip3 install pyspark==3.1.2 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
else
    echo 'Currently only python 3.6, 3.7 , 3.8 and 3.9 are supported, in case you need a different version please open an issue at https://github.com/IBM/claimed/issues'
    exit -1
fi

37
Starting installation...
Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)


## Import libraries

In [3]:
from pyspark import SparkContext, SparkConf, SQLContext
import os
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark2pmml import PMMLBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
import logging
import shutil
import site
import sys
import wget
import re

## Read the parquet file & Convert the parquet file to CSV format

In [4]:
if sys.version[0:3] == '3.9':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.8':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.7':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
elif sys.version[0:3] == '3.6':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
else:
    raise Exception('Currently only python 3.6 , 3.7, 3,8 and 3.9 is supported, in case '
                    'you need a different version please open an issue at '
                    'https://github.com/IBM/claimed/issues')

In [5]:
data_parquet = os.environ.get('data_parquet',
                              'data.parquet')  # input file name (parquet)
master = os.environ.get('master',
                        "local[*]")  # URL to Spark master
model_target = os.environ.get('model_target',
                              "model.xml")  # model output file name
data_dir = os.environ.get('data_dir',
                          '../../data/')  # temporary directory for data
input_columns = os.environ.get('input_columns',
                               '["x", "y", "z"]')  # input columns to consider

In [6]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [7]:
conf = SparkConf().setMaster(master)
#if sys.version[0:3] == '3.6' or sys.version[0:3] == '3.7':
conf.set("spark.jars", 'jpmml-sparkml-executable-1.5.12.jar')

sc = SparkContext.getOrCreate(conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

23/03/22 16:44:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Load the CSV file into a dataframe

In [8]:
df = spark.read.parquet(data_dir + data_parquet)

In [9]:
# register a corresponding query table
df.createOrReplaceTempView('df')

In [10]:
from pyspark.sql.types import DoubleType
df = df.withColumn("x", df.x.cast(DoubleType()))
df = df.withColumn("y", df.y.cast(DoubleType()))
df = df.withColumn("z", df.z.cast(DoubleType()))

## Create a 80-20 training and test split with seed=1

In [11]:
splits = df.randomSplit([0.8, 0.2],1)
df_train = splits[0]
df_test = splits[1]

In [12]:
indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=eval(input_columns),
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

## Train a Random Forest model with different hyperparameters
Hyper parameters:
  - number of trees : {10, 20}
  - maximum depth : {5, 7} 
  - use random seed = 1 wherever needed

In [13]:
def trainModelRandomForest(num_trees=10, max_depth=5, seed=1, df_train=''):
    print(f'Training Model by (number of trees={num_trees}, maximum depth= {max_depth})')
    RF = RandomForestClassifier(numTrees=num_trees, maxDepth=max_depth, seed=seed)
    pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, RF])
    model = pipeline.fit(df_train)
    prediction = model.transform(df_train)
    binEval = MulticlassClassificationEvaluator(). \
    setMetricName("accuracy"). \
    setPredictionCol("prediction"). \
    setLabelCol("label")
    acc = binEval.evaluate(prediction)
    print(f'The accuracy: {acc}\n')
    print('-'*100)
    return (acc, num_trees, max_depth, seed)

In [14]:
# Hyperparameters are training parameters
num_trees = [10, 20]
max_depth = [5,7]
performaceList = []
for i in range(len(num_trees)):
    for j in range(len(max_depth)):
        performaceList.append(trainModelRandomForest(num_trees[i], max_depth[j], 1, df_train))

Training Model by (number of trees=10, maximum depth= 5)


The accuracy: 0.44382182942402043

----------------------------------------------------------------------------------------------------
Training Model by (number of trees=10, maximum depth= 7)


The accuracy: 0.4634804086296122

----------------------------------------------------------------------------------------------------
Training Model by (number of trees=20, maximum depth= 5)


The accuracy: 0.44449391760198936

----------------------------------------------------------------------------------------------------
Training Model by (number of trees=20, maximum depth= 7)


[Stage 99:=================================================>        (6 + 1) / 7]

The accuracy: 0.46789378766494166

----------------------------------------------------------------------------------------------------


In [15]:
import numpy as np
performList = np.array(performaceList)
indexMax = performList[:,0].argmax(axis=0)
hAcc, hnum_trees, hmax_depth, hseed = performaceList[indexMax]
print("The highest accuracy is {} of training model by (number of trees={}, maximum depth={}, random seed={})".format(hAcc, hnum_trees, hmax_depth, hseed))

The highest accuracy is 0.46789378766494166 of training model by (number of trees=20, maximum depth=7, random seed=1)
